# Modelo Base en Pyomo – Caso 1: CVRP Estándar
Este notebook implementa el modelo base de ruteo de vehículos con capacidad y autonomía para LogistiCo usando Pyomo.

In [7]:
!pip install pandas
!pip install matplotlib
!pip install seaborn



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
!pip install pyomo[solvers]


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
%pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [10]:
!apt-get install -y glpk-utils

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
solver = SolverFactory('glpk')

## 1. Cargar Datos

In [13]:
# Cargar archivos CSV
vehicles = pd.read_csv("Datos\Vehicles.csv")
stations = pd.read_csv("Datos\stations.csv")  # No se usa en Caso 1
depots = pd.read_csv("Datos\depots.csv")
clients = pd.read_csv("Datos\clients.csv")

# Mostrar los datos cargados
display(vehicles)
display(depots)
display(clients)


<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_17384\2429352502.py:2: SyntaxWarning: invalid escape sequence '\V'
  vehicles = pd.read_csv("Datos\Vehicles.csv")
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_17384\2429352502.py:3: SyntaxWarning: invalid escape sequence '\s'
  stations = pd.read_csv("Datos\stations.csv")  # No se usa en Caso 1
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_17384\2429352502.py:4: SyntaxWarning: invalid escape sequence '\d'
  depots = pd.read_csv("Datos\depots.csv")
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_17384\2429352502.py:5: SyntaxWa

,VehicleID,Type,Capacity,Range
0,1,Large Truck,80.0,1720
1,2,Medium Truck,60.0,1510
2,3,Medium Truck,50.0,1300
3,4,Small Truck,40.0,1100
4,5,Small Truck,30.0,870


,DepotID,Latitude,Longitude,Updated
0,1,10.963889,-74.796387,Yes


,ClientID,City/Municipality,Demand,LocationID
0,1,Bogotá,30.0,2
1,2,Medellín,25.0,3
2,3,Cali,22.0,4
3,4,Cartagena,18.0,5
4,5,Cúcuta,15.0,6
5,6,Bucaramanga,17.0,7
6,7,Pereira,12.0,8
7,8,Santa Marta,10.0,9
8,9,Ibagué,11.0,10
9,10,Manizales,9.0,11


## 2. Definición de Conjuntos y Parámetros

In [14]:
# Extraer índices
V = list(vehicles["VehicleID"])
N = list(clients["LocationID"])
N_depot = list(depots["DepotID"])  # <- esta línea corregida

# Crear nodos con demandas (0 para depósitos)
demanda = dict(zip(clients["LocationID"], clients["Demand"]))
for depot_id in N_depot:
    demanda[depot_id] = 0

# Parámetros de capacidad
capacidad = dict(zip(vehicles["VehicleID"], vehicles["Capacity"]))
autonomia = dict(zip(vehicles["VehicleID"], vehicles["Range"]))# asumiendo como proxy

# Crear una matriz de distancias artificial entre nodos (euclidiana o constante por ahora)
from itertools import product
distancias = {(i, j): 1 if i != j else 0 for i, j in product(demanda.keys(), repeat=2)}


## 3. Definición del Modelo en Pyomo

In [15]:
model = pyo.ConcreteModel()

model.N = pyo.Set(initialize=demanda.keys())
model.V = pyo.Set(initialize=V)
model.A = pyo.Set(initialize=distancias.keys(), dimen=2)

model.x = pyo.Var(model.A, model.V, domain=pyo.Binary)
model.q = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)

model.obj = pyo.Objective(
    expr=sum(distancias[i, j] * model.x[i, j, v] for (i, j) in model.A for v in model.V),
    sense=pyo.minimize
)


## 4. Restricciones del Modelo

In [16]:
def flujo_entrada(model, j):
    if demanda[j] == 0:
        return pyo.Constraint.Skip
    return sum(model.x[i, j, v] for (i, j2) in model.A if j2 == j for v in model.V) == 1
model.restr_entrada = pyo.Constraint(model.N, rule=flujo_entrada)

def flujo_salida(model, i):
    if demanda[i] == 0:
        return pyo.Constraint.Skip
    return sum(model.x[i, j, v] for (i2, j) in model.A if i2 == i for v in model.V) == 1
model.restr_salida = pyo.Constraint(model.N, rule=flujo_salida)

def capacidad_vehiculo(model, v):
    return sum(demanda[j] * model.x[i, j, v] for (i, j) in model.A if j in demanda) <= capacidad[v]
model.restr_capacidad = pyo.Constraint(model.V, rule=capacidad_vehiculo)


## 5. Solución y Resultados

In [18]:
# Ensure the 'cbc' solver is installed
%pip install pyomo[solvers]
!apt-get install -y glpk-utils
!apt-get install -y coinor-cbc

solver = SolverFactory('glpk', executable="C:\\glpk-4.65\\w64\\glpsol.exe")
  # Cambiar a 'cbc' o 'gurobi' si está disponible
results = solver.solve(model, tee=True)

for (i, j) in model.A:
    for v in model.V:
        if model.x[i, j, v].value is not None and model.x[i, j, v].value > 0.5:
            print(f"Vehículo {v} viaja de {i} a {j}")


Note: you may need to restart the kernel to use updated packages.


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\MARIAN~1\AppData\Local\Temp\tmpp_t8hivx.glpk.raw --wglp
 C:\Users\MARIAN~1\AppData\Local\Temp\tmpwczd3tnk.glpk.glp --cpxlp C:\Users\MARIAN~1\AppData\Local\Temp\tmpgmgab9yu.pyomo.lp
Reading problem data from 'C:\Users\MARIAN~1\AppData\Local\Temp\tmpgmgab9yu.pyomo.lp'...
C:\Users\MARIAN~1\AppData\Local\Temp\tmpgmgab9yu.pyomo.lp:5429: warning: lower bound of variable 'x1053' redefined
C:\Users\MARIAN~1\AppData\Local\Temp\tmpgmgab9yu.pyomo.lp:5429: warning: upper bound of variable 'x1053' redefined
33 rows, 1120 columns, 3150 non-zeros
1120 integer variables, all of which are binary
6549 lines were read
Writing problem data to 'C:\Users\MARIAN~1\AppData\Local\Temp\tmpwczd3tnk.glpk.glp'...
5389 lines were written
GLPK Integer Optimizer, v4.65
33 rows, 1120 columns, 3150 non-zeros
1120 integer variables, all of which are binary
Preprocessing...
33 rows, 1120 columns, 3150 non-zeros
1120 integer va

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
